<a href="https://colab.research.google.com/github/amyush/FrequencyDomain_Watermarking/blob/main/watermarking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [2]:
!git clone https://github.com/amyush/FrequencyDomain_Watermarking.git

Cloning into 'FrequencyDomain_Watermarking'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [3]:
import numpy as np
import cv2
import math
from PIL import Image
import matplotlib.pyplot as plt
from PIL import Image as im
import math

# **Custom Functions**

In [4]:
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

def binaryToString(a):
  return chr(binaryToInt(a))

def integerToBinary(a):
  return int(bin(a)[2:])

def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

def calculatePSNR(originalImage, encodedImage):
  error = np.sum((originalImage.astype('float') - encodedImage.astype('float')) ** 2)
  error /= float(originalImage.shape[0] * originalImage.shape[1]);
  if error == 0:
      return 100
  PIXEL_MAX = 255.0
  return 20 * math.log10(PIXEL_MAX / math.sqrt(error))

def computeDCT(image):
    dct = np.zeros(shape=image.shape)
    dim = 1
    if(len(image.shape) > 2):
        dim = image.shape[2]

    for i in range(dim):
        dct[i] = cv2.dct(image[i])

    return dct 

def computeIDCT(dctImage):
    idct = np.zeros(shape=dctImage.shape)
    dim = 1
    if(len(dctImage.shape) > 2):
        dim = dctImage.shape[2]

    for i in range(dim):
        idct[i] = cv2.idct(dctImage[i])

    for d in range(dim):
        for i in range(idct.shape[0]):
            for j in range(idct.shape[1]):
                idct[i][j][d] = abs(math.sqrt(idct[i][j][d].real**2 + idct[i][j][d].imag**2))

    return idct 


# **Steganography**

## **Hiding the data**

In [55]:
def encodingImage(imageName, key, hiddenImageName):
    imgPath ="FrequencyDomain_Watermarking/"
    delimeter = "#@#@#@"

    inputImage = cv2.imread(imgPath + imageName, cv2.IMREAD_UNCHANGED)
    # inputImage = inputImage.resize(2048,2048,3)
    waterMarkImage = cv2.imread(imgPath + hiddenImageName, cv2.IMREAD_UNCHANGED)
    waterMarkImage = waterMarkImage.resize(8,8)

    binKey = stringToBinary(key)
    lenBinKey = len(binKey)

    height = inputImage.shape[0]
    width = inputImage.shape[1]
    dim = 1

    if(len(inputImage.shape) > 2):
        dim = inputImage.shape[2]

    flatOrigImage = np.ravel(np.array(inputImage, dtype=np.float))
    flatWatermarkImage = np.ravel(np.array(waterMarkImage, dtype=np.float))

    dctOrigImage = cv2.dct(flatOrigImage)

    outputImage = dctOrigImage

    index = -1
    count = 0
    pixel = 0

    for i in range(len(dctOrigImage)):
        index = index + 1;
        if((int)(binKey[index%lenBinKey]) == 1):
            if(count < flatWatermarkImage.shape[0]):
                outputImage[i] = flatWatermarkImage[count]
                count = count + 1;

    idctArray = cv2.idct(outputImage)
    resultArray = np.reshape(idctArray, (height, width, dim)).astype(np.uint8)
    resImage = cv2.imwrite('res.jpg', resultArray)
    return inputImage, resImage
    

In [57]:
# coverFileName = input("Enter the cover filename: (eg- Cover_1.png or Cover_2.png)")
# key = input("Enter the key:")
# hiddenImageName = input("Enter the name of the image to be hidden:")

coverFileName = 'cover.jpg'
key = 'Appawdawdawdawdle'
hiddenImageName = 'logo.png'


inputImage, outputImage = encodingImage(coverFileName, key, hiddenImageName)

# plt.subplot(1,2,1),plt.imshow(cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)), plt.title("Input Image")
# plt.subplot(1,2,2),plt.imshow(outputImage), plt.title("Output Image") 
# plt.subplot(1,3,3),plt.imshow(cv2.cvtColor(idct, cv2.COLOR_BGR2RGB)), plt.title("Output Image") 

# **Results**

In [ ]:
def encode(img_path, wm_path, res_path, alpha):
    img = cv2.imread(img_path)
    img_f = np.fft.fft2(img)
    height, width, channel = np.shape(img)
    watermark = cv2.imread(wm_path)
    wm_height, wm_width = watermark.shape[0], watermark.shape[1]
    x, y = range(height / 2), range(width)
    random.seed(height + width)
    random.shuffle(x)
    random.shuffle(y)
    tmp = np.zeros(img.shape)
    for i in range(height / 2):
        for j in range(width):
            if x[i] < wm_height and y[j] < wm_width:
                tmp[i][j] = watermark[x[i]][y[j]]
                tmp[height - 1 - i][width - 1 - j] = tmp[i][j]
    res_f = img_f + alpha * tmp
    res = np.fft.ifft2(res_f)
    res = np.real(res)
    cv2.imwrite(res_path, res, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [ ]:
plt.subplot(1,3,1),plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB)), plt.title("Input Image")
